In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.feature_selection import *

In [ ]:
# Function to get column names
def get_column_names():
    """
    Get list of column names from UCI provided .names file
    Input:
        N/A
    Output:
        A list of column names for the dataset
    """
    file = open(os.path.join(INPUT_DIR, 'spambase.names'), 'r')
    result = []
    while True:
        line = file.readline()
        if (not line):
            break
        else:
            if (line.startswith('word_freq_') or (line.startswith('char_freq_')) or (line.startswith('capital_run'))):
                line = line.split(':')
                result.append(line[0])
    result.append('Target')
    return result

In [ ]:
# Function to plot ROC curve
def plot_roc_curve(fpr, tpr, title=None):
    plt.title(title)
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.show()

In [ ]:
# Implementing a naive Bayes
# We want to implement a regularized form of naive Bayes where a missing
# feature does not hinder the computation

# A) Read the input in the input directory and do test_train_split within
# the notebook (5 points)

# Get working directory path
PWD = os.getcwd()
# Get input directory path
INPUT_DIR = os.path.join(PWD, 'input')
# Get dataset path
DATASET_PATH = os.path.join(INPUT_DIR, 'spambase.csv')
# Get column names
col_names = get_column_names()
# Read data
data = pd.read_csv(DATASET_PATH, names=col_names)
# Get 'Target' column
labels = pd.DataFrame(data.pop('Target'))
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=2)

In [ ]:
# Select 'k' amount of features and categorize them
def select_k_best_features(X, y, k):
    feature_names = X.columns.tolist()
    selector = SelectKBest(chi2, k=k)
    selector.fit(X, y)
    mask = selector.get_support()
    new_features = []
    for boolean, feature in zip(mask, feature_names):
        if boolean:
            new_features.append(feature)
    X_out = selector.transform(X)
    return pd.DataFrame(X_out, index=X.index, columns=new_features)

def select_k_bins(X, k):
    est = KBinsDiscretizer(n_bins=k, encode='ordinal', strategy='uniform')
    X_out = est.fit_transform(X)
    return pd.DataFrame(X_out, index=X.index, columns=X.columns)
temp = select_k_best_features(X_train, y_train, 10)
temp = select_k_bins(temp, 3)
temp.head()

,word_freq_remove,word_freq_free,word_freq_your,word_freq_hp,word_freq_hpl,word_freq_george,char_freq_!,capital_run_length_average,capital_run_length_longest,capital_run_length_total
3051,0.00,0.0,0.61,0.00,0.00,0.00,0.327,1.068,2.0,47.0
3349,0.00,0.0,0.00,0.00,0.00,0.00,0.000,2.000,4.0,6.0
3529,0.00,0.0,0.00,0.00,0.00,0.00,0.000,2.500,4.0,5.0
1505,0.23,0.0,3.30,0.00,0.00,0.00,1.289,6.529,276.0,666.0
2718,0.00,0.0,0.64,1.28,0.64,1.92,0.225,1.902,12.0,78.0


In [ ]:
# I will use the top 10 and bottom 10 columns based off their positive and
# negative correlation to the label column
top_and_bottom_10_col_names = [
    'word_freq_your',
    'word_freq_000',
    'word_freq_remove',
    'char_freq_$',
    'char_freq_!',
    'word_freq_business',
    'word_freq_you',
    'word_freq_our',
    'capital_run_length_total',
    'word_freq_free',
    'word_freq_technology',
    'word_freq_re',
    'word_freq_85',
    'word_freq_edu',
    'word_freq_650',
    'word_freq_labs',
    'word_freq_1999',
    'word_freq_george',
    'word_freq_hpl',
    'word_freq_hp'
]

# New column bins thru visual analysis
top_and_bottom_10_bins = [
    [0, 3, 6, 12],
    [0, 1, 6],
    [0, 3, 8],
    [0, 1, 2, 7],
    [0, 1, 4, 33],
    [0, 1, 2, 8],
    [0, 1, 5, 19],
    [0, 2, 5, 11],
    [0, 10, 15842],
    [0, 2, 21],
    [0, 1, 5, 8],
    [0, 10, 22],
    [0, 1, 10, 21],
    [0, 1, 10 , 23],
    [0, 1, 3, 10],
    [0, 1, 3, 6],
    [0, 1, 3, 7],
    [0, 1, 2, 34],
    [0, 1, 17],
    [0, 1, 21]
]

# New column discretized labels (chosen completely by my discretion)
top_and_bottom_10_labels = [
    ['low', 'medium', 'high'],
    ['low', 'high'],
    ['low', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'high'],
    ['low', 'high'],
    ['low', 'medium', 'high'],
    ['medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'medium', 'high'],
    ['low', 'high'],
    ['low', 'high']
]
# Change X_train and X_test columns
X_train = X_train[top_and_bottom_10_col_names]
X_test = X_test[top_and_bottom_10_col_names]

# Discretize all continuous columns
for i in range(len(top_and_bottom_10_col_names)):
    # Get current column name
    col_name = top_and_bottom_10_col_names[i]
    # Get associated bins for column
    col_bins = top_and_bottom_10_bins[i]
    # Get associated labels for bins
    col_label = top_and_bottom_10_labels[i]
    # Discretize X_train
    X_train[col_name] = pd.cut(X_train[col_name], col_bins, labels=col_label, right=False)
    # Discretize X_test
    X_test[col_name] = pd.cut(X_test[col_name], col_bins, labels=col_label, right=False)

In [ ]:
# Code to implement naive Bayes with any exception handling.
class NaiveBayes:
    def __init__(self):
        """
        Creates a NaiveBayes object
        Input:
            N/A
        Output:
            N/A
        Initialization:
            self.word_proba: Initialize a DataFrame which keeps attribute values as columns
and probabilites of that attribute value being spam as one row and ham as another (i.e., 
P(word_freq_free_a_i|spam): word_freq_free_a_i, P(word_freq_free_a_i|not spam): word_freq_free_a_i)
            self.pos_prior: Initialize prior probabilities of positive instances to None
            self.neg_prior: Initialize prior probabilities of negative instances to None
            self.true_neg: Initialize true negative value
            self.true_pos: Initialize true positive value
            self.false_neg: Initialize false negative value
            self.false_pos: Initialize false positive value
            self.confusion_matrix: Initialize beginning of 2D matrix (a.k.a confusion matrix)
            self.preds: Initialize 1D array of length equal to testing set which holds predictions
            self.pred_probas: Initialize beginning of 2D matrix containing 1D arrays a length k
where k equals the set of values in label column and they contain the probability that the instance
belongs to class label k (range of values for k, 0 <= i <= k-1)
        """
        self.word_proba = pd.DataFrame()
        self.pos_prior = None
        self.neg_prior = None
        self.true_neg = 0
        self.true_pos = 0
        self.false_neg = 0
        self.false_pos = 0
        self.confusion_matrix = []
        self.preds = []
        self.pred_probas = []
    
    def fit(self, X_train, y_train):
        """
        Calculates prior probabilities, and probabilites of every attribute and their values
        given all k class labels (i.e., binary classification, k = 2)
        Input:
            X_train: Dataset to train on
            y_train: X_train's corresponding labels
        Output:
            N/A
            Populates a DataFrame with probabilities explained above
        """
        # If X_train is not an instance of a DataFrame
        if (not(isinstance(X_train, pd.DataFrame))):
            raise Exception('X_train is not of type \'pd.DataFrame\'')
            
        if (not(isinstance(y_train, pd.DataFrame)) and not(isinstance(y_train, pd.Series))):
            raise Exception('y_train is not of type \'pd.DataFrame\' or \'pd.Series\'')
            
        # Get the column name of the class column 
        class_col_label = y_train.columns[0]
        # Get number of spam and ham instances
        class_freq = y_train[class_col_label].value_counts()
        num_spam = class_freq[1]
        num_ham = class_freq[0]
        # Get prior probabilities
        self.pos_prior, self.neg_prior = self._class_priors(num_spam, num_ham)
        # Get conditional probabilities
        self._get_conditional_probs(X_train, y_train, num_spam, num_ham)
    
    def predict(self, X_test):
        """
        Returns a vector of size m x 1 where m equals the length of X_test. The
        vector consists of predictions on X_test with k unique values (k being the
        number of classes we're dealing with)
        
        Input:
            X_test: The testing set to make class predictions on
        Output:
            self.preds: A vector of class predictions our model made
            self.true_neg: Counts amounts of true negatives and assigns them to self.true_neg
            self.true_pos: Counts amounts of true positives and assigns them to self.true_pos
            self.false_neg: Counts amounts of false negatives and assigns them to self.false_neg
            self.false_pos: Counts amounts of false positives and assigns them to self.false_pos
        """
        # Exception handling
        if (not(isinstance(X_test, pd.DataFrame))):
            raise Exception('X_test is not of type \'pd.DataFrame\'')
            
        # Predictions data structure was already filled
        if (len(self.preds) != 0):
            self.preds = []
        
        # Get column names of 'X_test'
        col_names = X_test.columns.tolist()
        # Iterate over every row in 'X_test'
        for index, row in X_test.iterrows():
            # Calculate the Naive Bayes Classifier formula for spam and ham class labels
            ham_proba = self.neg_prior
            spam_proba = self.pos_prior
            # Iterate over every column to get correct column name to look in 'self.word_proba'
            for col_name in col_names:
                # Get value in current column name and current row in 'X_test'
                row_col_value = row[col_name]
                # Concatenate to get correct column name for 'self.word_proba'
                proba_col_name = str(col_name + '_' + row_col_value)
                # Get probability P(a_i|Not Spam)
                row_col_val_ham = self.word_proba[proba_col_name][0]
                # Get probability P(a_i|Spam)
                row_col_val_spam = self.word_proba[proba_col_name][1]
                # Multiply both probabilities with respective 'ham_proba' and 'spam_proba'
                ham_proba *= row_col_val_ham
                spam_proba *= row_col_val_spam
            # See which accumulated probability is larger
            if (ham_proba > spam_proba):
                # 'ham_proba' is greater than 'spam_proba', so append 0 to 'self.preds'
                self.preds.append(0)
            else:
                # 'spam_proba' is greater than 'ham_proba', so append 0 to 'self.preds'
                self.preds.append(1)
        return self.preds
    
    def predict_probas(self, X_test):
        """
        Returns a matrix of size m x k where m equals the length of X_test and k
        is the number of classes we are dealing with (i.e., for binary classification
        k = 2). This matrix consists of probabilities for each instance of X_test 
        being belonging to each class k
        
        Input:
            X_test: The testing set to make prediction of probabilities on
        Output:
            self.pred_probas: A matrix of size m x k which consists of probabilites
for each instance of X_test belonging to class k
        """
        # Exception handling
        if (not(isinstance(X_test, pd.DataFrame))):
            raise Exception('X_test is not of type \'pd.DataFrame\'')
            
        # Probabilities data structure was already filled
        if (len(self.pred_probas) != 0):
            self.pred_probas = []
        
        # Get column names of 'X_test'
        col_names = X_test.columns.tolist()
        # Iterate over every row in 'X_test'
        for index, row in X_test.iterrows():
            # Probabilities to store each respective probability in
            probas = []
            # Calculate the Naive Bayes Classifier formula for spam and ham class labels
            ham_proba = self.neg_prior
            spam_proba = self.pos_prior
            # Iterate over every column to get correct column name to look in 'self.word_proba'
            for col_name in col_names:
                # Get value in current column name and current row in 'X_test'
                row_col_value = row[col_name]
                # Concatenate to get correct column name for 'self.word_proba'
                proba_col_name = str(col_name + '_' + row_col_value)
                # Get probability P(a_i|Not Spam)
                row_col_val_ham = self.word_proba[proba_col_name][0]
                # Get probability P(a_i|Spam)
                row_col_val_spam = self.word_proba[proba_col_name][1]
                # Multiply both probabilities with respective 'ham_proba' and 'spam_proba'
                ham_proba *= row_col_val_ham
                spam_proba *= row_col_val_spam
            # Get sums of both probabilities
            ham_spam_proba_sum = ham_proba + spam_proba
            # Get actual percentage value of 'ham_proba' and 'spam_proba' and store them in 'probas'
            ham_proba /= ham_spam_proba_sum
            spam_proba /= ham_spam_proba_sum
            # Append 'ham_proba' and 'spam_proba' to 'probas'
            probas.append(ham_proba)
            probas.append(spam_proba)
            # Append array to 'self.pred_probas' array
            self.pred_probas.append(probas)
        return self.pred_probas
                
    def get_metrics(self, predictions, y_test):
        """
        Initialize metrics of this class such as self.true_neg, self.true_pos, 
self.false_neg, self.false_pos, and self.confusion_matrix. Also return such metrics
and all this being tested against the ground truths of y_test
        Input:
            predictions: The predictions previously acquired by running 'predict' method
            y_test: Ground truths to be compared against
        Output:
            self.true_neg: The true negative value
            self.true_pos: The true positive value
            self.false_neg: The false negative value
            self.false_pos: The false positive value
            self.confusion_matrix: The confusion matrix comprised of TN, TP, FN, FP
        """
        # Handling exceptions
        if (len(predictions) == 0):
            raise Exception('You have not run \'predict\' method previously and do not have a list of predictions')
        
        if (not(isinstance(y_test, pd.DataFrame))):
            raise Exception('X_test is not of type \'pd.DataFrame\'')
        
        # Get class column label
        class_col_name = y_test.columns[0]
        # Select label column and convert the data structure to a list
        labels = y_test[class_col_name].tolist()
        # Iterate over both lists checking each entry against each other
        for i in range(len(predictions)):
            if (predictions[i] == 0 and labels[i] == 0):
                # True negative case
                self.true_neg += 1
            elif (predictions[i] == 1 and labels[i] == 1):
                # True positive case
                self.true_pos += 1
            elif (predictions[i] == 0 and labels[i] == 1):
                # False negative case
                self.false_neg += 1
            elif (predictions[i] == 1 and labels[i] == 0):
                # False positive case
                self.false_pos += 1
        # Populate 'self.confusion_matrix'
        self.confusion_matrix = [[self.true_neg, self.false_pos], [self.false_neg, self.true_pos]]
        return self.true_neg, self.true_pos, self.false_neg, self.false_pos, self.confusion_matrix
    
    def _class_priors(self, num_spam, num_ham):
        """
        Input:
            num_spam: Number of spam messages
            num_ham: Number of ham messages
        Output:
            pos_prior: The prior probabilities of the positive examples given y
            neg_prior: The prior probabilities of the negative examples given y
        """ 
        total_len = num_spam + num_ham
        pos_prior = num_spam / total_len
        neg_prior = num_ham / total_len
        return pos_prior, neg_prior
    
    def _get_conditional_probs(self, X, y, num_spam, num_ham):
        """
        Populates self.word_proba with conditional probabilities of every attributes
        values according to different set of class values
        """
        # Copy the X dataset
        data = X.copy()
        # Get class column label
        class_col_name = y.columns[0]
        # Get list of unique class values
        class_uniques = list(y[class_col_name].unique())
        # Concatenate the class label column to the copy
        data[class_col_name] = y[class_col_name]
        # Initialize 'self.word_proba' with correct column and row labels
        self._initialize_word_proba_ds(X, y)
        # Get column names of 'data'
        col_names = data.columns.tolist()
        # Iterate over every column calculating conditional probabilities
        for col_name in col_names:
            # Disregard the 'class_col_name' column
            if (col_name == class_col_name):
                continue
                
            # Get attribute values for current attribute/column
            uniques = list(data[col_name].unique())
            # Iterate over the set of attribute values
            for attrib_value in uniques:
                # Select column name to populate wrt 'self.word_proba' column naming
                proba_col_name = str(col_name + '_' + attrib_value)
                # Iterate over set of class values to get probability wrt a class label
                for class_value in class_uniques:
                    # Get frequency of 'attrib_value'
                    attrib_value_freq = len(data.loc[(data[col_name] == attrib_value) & (data[class_col_name] == class_value), col_name])
                    # Put frequency divided by number of spam or ham instances into appropriate column and row
                    if (class_value == 0):
                        # 'class_value' is equal to ham (0) so divide by 'num_ham'
                        self.word_proba[proba_col_name][class_value] = (attrib_value_freq / num_ham)
                    else:
                        # 'class_value' is equal to spam (1) so divide by 'num_spam'
                        self.word_proba[proba_col_name][class_value] = (attrib_value_freq / num_spam)
        
    def _initialize_word_proba_ds(self, X, y):
        """
        Initializes self.word_proba where all entries are zero and row and columns
        are labeled correctly
        
        Input:
            X: The dataset we train on
            y: The associated labels
        Output:
            Initializes our data structure that holds are probabilities
        """
        # Get X's columns
        col_names = X.columns.tolist()
        # Iterate over all columns in 'X'
        for col_name in col_names:
            # Get unique values for that column
            uniques = list(X[col_name].unique())
            # Iterate over all the unique values of current attribute
            for attrib_value in uniques:
                # Create column name by concatenating 'col_name' and 'attrib_value'
                new_col_name = str(col_name + '_' + attrib_value)
                # Append new column
                self.word_proba[new_col_name] = None
              
        # Get class column label
        y_col_name = y.columns[0]
        # Get unique values for class label column
        y_uniques = y[y_col_name].unique()
        # Populate list of null values equal to the number of columns in 'self.word_proba'
        dummy_values = []
        for i in range(len(self.word_proba.columns)):
            dummy_values.append(0)
        # Create list of Series Pandas objects
        list_of_series = []
        # Iterate over unique values in class label column
        for unique in y_uniques:
            # Create Series object
            series = pd.Series(dummy_values, index=self.word_proba.columns)
            # Add 'series' to list of Series objects
            list_of_series.append(series)
        # Append the k number of unique class values to 'self.word_proba' data structure
        self.word_proba = self.word_proba.append(list_of_series, ignore_index=True)

In [ ]:
# Fit the model to some data
nb = NaiveBayes()
nb.fit(X_train, y_train)

In [ ]:
# Visual display of training phase of the model
nb.word_proba.head()

In [ ]:
# Output a confusion matrix. Compute accuracy, precision, recall and obtain an AUC plot of the classifier

# Get list of predictions
preds = nb.predict(X_test)
# Get 2D Matrix of probabilities of predictions
pred_probas = nb.predict_probas(X_test)
# Convert to numpy.ndarray
np_pred_probas = np.array(pred_probas, np.float64)
# Get spam probabilities
pos_probas = np_pred_probas[:, 1]
# Get confusion matrix, precision, and recall
true_neg, true_pos, false_neg, false_pos, confusion_matrix = nb.get_metrics(preds, y_test)
print(f'True negatives: {true_neg}')
print(f'True positive: {true_pos}')
print(f'False negatives: {false_neg}')
print(f'False positive: {false_pos}')
print(f'Confusion matrix: {confusion_matrix}')
print('--------------------------------------')
print(f'Accuracy: {(true_neg + true_pos)/(true_neg + true_pos + false_neg + false_pos)}')
print(f'Precision: {(true_pos)/(true_pos + false_pos)}')
print(f'Recall: {(true_pos)/(true_pos + false_neg)}')
print('--------------------------------------')

# Plot the two AUCs
nb_fpr, nb_tpr, nb_thresholds = roc_curve(y_test, pos_probas)
plot_roc_curve(nb_fpr, nb_tpr, "Naive Bayes")